In [29]:
import gc
import os
import time
import logging
import datetime
import warnings
import numpy as np
import pandas as pd
from tqdm import tqdm
import seaborn as sns
import xgboost as xgb
import lightgbm as lgb
from scipy import stats
from scipy.signal import hann
from tqdm import tqdm_notebook
import matplotlib.pyplot as plt
from scipy.signal import hilbert
from scipy.signal import convolve
from sklearn.svm import NuSVR, SVR
from catboost import CatBoostRegressor
from sklearn.kernel_ridge import KernelRidge
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_absolute_error
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold,StratifiedKFold, RepeatedKFold
from sklearn.ensemble import RandomForestRegressor
warnings.filterwarnings("ignore")

In [4]:
fulldf=pd.read_csv('D:/NINI data/data/fulldf.csv',index_col=0)

In [5]:
fulldf.head()

,label,mean_full,sd_full,q0_full,q0.01_full,q0.05_full,q0.1_full,q0.25_full,q0.5_full,q0.75_full,...,sd_roll_sd_1500,q0_roll_sd_1500,q0.25_roll_sd_1500,q0.75_roll_sd_1500,q1_roll_sd_1500,ar1_roll_sd_1500,kur_roll_sd_1500,skew_roll_sd_1500,min_roll_sd_1500,max_roll_sd_1500
1,1.430797,4.884113,5.101106,-98,-8.0,-2.0,0.0,3,5.0,7.0,...,2.656865,2.617320,2.995398,4.788156,25.824677,0.999997,30.344736,4.762917,2.617320,25.824677
2,1.411696,4.746800,6.173383,-154,-9.0,-2.0,0.0,2,5.0,7.0,...,4.043365,2.617320,2.987012,4.789842,32.139798,0.999995,28.472282,5.088456,2.617320,32.139798
3,1.391499,4.725767,6.588824,-154,-11.0,-2.0,0.0,2,5.0,7.0,...,4.337875,2.592122,2.966430,5.295588,32.139798,0.999995,20.393856,4.228071,2.592122,32.139798
4,1.372297,4.753000,5.715036,-115,-11.0,-2.0,0.0,2,5.0,7.0,...,3.103912,2.592122,2.993000,5.308373,18.720802,0.999994,5.218347,2.257274,2.592122,18.720802
5,1.353196,4.906393,6.967397,-106,-15.0,-3.0,0.0,2,5.0,7.0,...,4.164771,2.619431,3.092818,6.069799,29.323379,0.999996,8.075628,2.556367,2.619431,29.323379


In [8]:
alltest=pd.read_csv('D:/NINI data/data/alltest.csv',index_col=0)

In [9]:
alltest.head()

,mean_full,sd_full,q0_full,q0.01_full,q0.05_full,q0.1_full,q0.25_full,q0.5_full,q0.75_full,q0.9_full,...,sd_roll_sd_1500,q0_roll_sd_1500,q0.25_roll_sd_1500,q0.75_roll_sd_1500,q1_roll_sd_1500,ar1_roll_sd_1500,kur_roll_sd_1500,skew_roll_sd_1500,min_roll_sd_1500,max_roll_sd_1500
1,4.491780,4.893690,-75,-8.0,-2,0.0,2.0,4,7,9,...,2.292318,2.570801,3.038639,4.433796,20.448716,0.999995,18.830727,3.678800,2.570801,20.448716
2,4.171153,5.922839,-140,-12.0,-2,-1.0,2.0,4,7,9,...,3.599575,2.588071,2.987653,4.551518,24.631268,0.999995,11.786130,3.322717,2.588071,24.631268
3,4.610260,6.946990,-193,-11.0,-2,0.0,2.0,5,7,9,...,5.023551,2.596135,2.943750,4.432930,43.596921,0.999995,30.025207,5.072505,2.596135,43.596921
4,4.531473,4.114147,-93,-5.0,-1,1.0,3.0,5,7,8,...,2.051264,2.489678,2.715904,3.245881,18.862115,0.999994,24.018188,4.336000,2.489678,18.862115
5,4.128340,5.797164,-147,-10.0,-2,0.0,2.0,4,6,9,...,3.683712,2.561236,2.857051,4.340566,28.828019,0.999996,18.076352,3.942079,2.561236,28.828019


In [36]:
train_X=fulldf.iloc[:,1:]
train_y=fulldf.iloc[:,0]

In [37]:
train_y.head()

1    1.430797
2    1.411696
3    1.391499
4    1.372297
5    1.353196
Name: label, dtype: float64

In [17]:
scaler = StandardScaler()
scaler.fit(train_X)
scaled_train_X = pd.DataFrame(scaler.transform(train_X), columns=train_X.columns)

In [18]:
scaled_test_X = pd.DataFrame(scaler.transform(alltest), columns=alltest.columns)

In [22]:
scaled_train_X.head()

,mean_full,sd_full,q0_full,q0.01_full,q0.05_full,q0.1_full,q0.25_full,q0.5_full,q0.75_full,q0.9_full,...,sd_roll_sd_1500,q0_roll_sd_1500,q0.25_roll_sd_1500,q0.75_roll_sd_1500,q1_roll_sd_1500,ar1_roll_sd_1500,kur_roll_sd_1500,skew_roll_sd_1500,min_roll_sd_1500,max_roll_sd_1500
0,1.417288,-0.174309,0.196545,0.239325,0.090408,0.182526,1.597236,0.948201,0.265415,0.645392,...,-0.187444,0.661393,0.096992,0.018557,-0.112766,1.154002,0.720015,0.609325,0.661393,-0.112766
1,0.880347,-0.049123,-0.011966,0.166802,0.090408,0.182526,-0.398281,0.948201,0.265415,0.645392,...,-0.021650,0.661393,0.067238,0.019562,-0.031079,0.016677,0.588726,0.852693,0.661393,-0.031079
2,0.798100,-0.000622,-0.011966,0.021756,0.090408,0.182526,-0.398281,0.948201,0.265415,0.645392,...,0.013566,0.286952,-0.005789,0.321140,-0.031079,-0.332303,0.022298,0.209482,0.286952,-0.031079
3,0.904591,-0.102634,0.133247,0.021756,0.090408,0.182526,-0.398281,0.948201,0.265415,0.645392,...,-0.133988,0.286952,0.088486,0.328764,-0.204657,-0.756343,-1.041749,-1.263855,0.286952,-0.204657
4,1.504410,0.043576,0.166758,-0.268336,-0.325657,0.182526,-0.398281,0.948201,0.265415,0.645392,...,-0.007133,0.692766,0.442644,0.782805,-0.067510,0.138341,-0.841408,-1.040257,0.692766,-0.067510


In [19]:
n_fold = 5
folds = KFold(n_splits=n_fold, shuffle=True, random_state=42)
train_columns = scaled_train_X.columns.values

In [38]:
params = {'num_leaves': 51,
         'min_data_in_leaf': 10, 
         'objective':'regression',
         'max_depth': -1,
         'learning_rate': 0.001,
         "boosting": "gbdt",
         "feature_fraction": 0.91,
         "bagging_freq": 1,
         "bagging_fraction": 0.91,
         "bagging_seed": 42,
         "metric": 'mae',
         "lambda_l1": 0.1,
         "verbosity": -1,
         "nthread": -1,
         "random_state": 42}

In [39]:
oof = np.zeros(len(scaled_train_X))
predictions = np.zeros(len(scaled_test_X))
feature_importance_df = pd.DataFrame()
#run model
for fold_, (trn_idx, val_idx) in enumerate(folds.split(scaled_train_X,train_y.values)):
    strLog = "fold {}".format(fold_)
    print(strLog)
    
    X_tr, X_val = scaled_train_X.iloc[trn_idx], scaled_train_X.iloc[val_idx]
    y_tr, y_val = train_y.iloc[trn_idx], train_y.iloc[val_idx]

    model = lgb.LGBMRegressor(**params, n_estimators = 20000, n_jobs = -1)
    model.fit(X_tr, 
              y_tr, 
              eval_set=[(X_tr, y_tr), (X_val, y_val)], 
              eval_metric='mae',
              verbose=1000, 
              early_stopping_rounds=500)
    oof[val_idx] = model.predict(X_val, num_iteration=model.best_iteration_)
    #feature importance
    fold_importance_df = pd.DataFrame()
    fold_importance_df["Feature"] = train_columns
    fold_importance_df["importance"] = model.feature_importances_[:len(train_columns)]
    fold_importance_df["fold"] = fold_ + 1
    feature_importance_df = pd.concat([feature_importance_df, fold_importance_df], axis=0)
    #predictions
    predictions += model.predict(scaled_test_X, num_iteration=model.best_iteration_) / folds.n_splits


fold 0
Training until validation scores don't improve for 500 rounds.
[1000]	training's l1: 2.03907	valid_1's l1: 2.22895
[2000]	training's l1: 1.76409	valid_1's l1: 2.09405
[3000]	training's l1: 1.61829	valid_1's l1: 2.05993
[4000]	training's l1: 1.50725	valid_1's l1: 2.0446
[5000]	training's l1: 1.41324	valid_1's l1: 2.03305
[6000]	training's l1: 1.32954	valid_1's l1: 2.02336
[7000]	training's l1: 1.25398	valid_1's l1: 2.01427
[8000]	training's l1: 1.18506	valid_1's l1: 2.00782
[9000]	training's l1: 1.1211	valid_1's l1: 2.00136
[10000]	training's l1: 1.06154	valid_1's l1: 1.99541
[11000]	training's l1: 1.00611	valid_1's l1: 1.99029
[12000]	training's l1: 0.954034	valid_1's l1: 1.98637
[13000]	training's l1: 0.905282	valid_1's l1: 1.98317
[14000]	training's l1: 0.859518	valid_1's l1: 1.98007
[15000]	training's l1: 0.81659	valid_1's l1: 1.97735
[16000]	training's l1: 0.776169	valid_1's l1: 1.97472
[17000]	training's l1: 0.738025	valid_1's l1: 1.9723
[18000]	training's l1: 0.701897	vali

In [24]:
submission = pd.read_csv('D:/NINI data/data/sample_submission.csv', index_col='seg_id')

In [40]:
submission.time_to_failure = predictions
submission.to_csv('D://NINI data/submission3.csv',index=True)

In [34]:
for fold_, (trn_idx, val_idx) in enumerate(folds.split(train_X,train_y.values)):
    strLog = "fold {}".format(fold_)
    print(strLog)
    X_tr, X_val = train_X.iloc[trn_idx], train_X.iloc[val_idx]
    y_tr, y_val = train_y.iloc[trn_idx], train_y.iloc[val_idx]
    model = RandomForestRegressor(n_estimators=1000,criterion='mae')
    model.fit(X_tr, y_tr) 
            
    print('mae',mean_absolute_error(train_y.iloc[val_idx],model.predict(train_X.iloc[val_idx])))
    

0it [00:00, ?it/s]

fold 0


KeyboardInterrupt: 